
<table align="center">
  <td align="center"><a target="_blank" href="https://colab.research.google.com/github/dhairya0907/Creepypasta-Text-Generator/blob/main/Scripts/Word_Level_Text_Generation_After_First_Time.ipynb">
        <img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Run in Google Colab</a></td>
  <td align="center"><a target="_blank" href="https://github.com/dhairya0907/Creepypasta-Text-Generator/blob/main/Scripts/Word_Level_Text_Generation_After_First_Time.ipynb">
        <img src="https://i.ibb.co/xfJbPmL/github.png"  height="70px" style="padding-bottom:5px;"  />View Source on GitHub</a></td>
</table>

In [ ]:
%%capture
!pip install wandb -qqq

In [ ]:
%load_ext tensorboard

In [ ]:
import tensorflow.compat.v2 as tf
from tensorflow.python.framework.ops import disable_eager_execution

tf.enable_v2_behavior()
disable_eager_execution()

In [ ]:
import os
import sys
import time
import wandb
import keras
import string
import datetime
import numpy as np

from wandb import AlertLevel
from tqdm import tqdm
from google.colab import drive
from wandb.keras import WandbCallback
from keras.layers import LSTM, Dense, Embedding

In [ ]:
drive.mount('/content/drive')

In [ ]:
wandb.login()

In [ ]:
Epochs = 50
Batch = 256
sequence_len = 20
diversity_random = 2 
predict_normal_len = 100
predict_random_len = 100
vocab_len_highest = 0 # len of the highest vocab in all the data
main_folder_name = "" # Folder name where the file is stored
file_string = "had , ? wine" # Any two words from the text file
file_name  = "" # File name on which you want traning to be done
last_file = "" # Last file in which you trained for incremantal learning

In [ ]:
folder_file = file_name.split("/")
last_folder_file = last_file.split("/")

In [ ]:
f = open("/content/drive/My Drive/path_to_file", "r")
text = f.read()
f.close()

In [ ]:
text = text.lower()
text = text.split()
vocab = set(text)
len(vocab)

In [ ]:
# Optional wandb

experiment_name = ""

wandb.init(project="", 
           name=file_name, 
           sync_tensorboard=True,
           group=experiment_name,
           notes="",
           tags=["", "", "","",""],)

artifact = wandb.Artifact('', type='dataset')
artifact.add_file("/content/drive/My Drive/path_to_file")
wandb.log_artifact(artifact)

wandb.config.update({"epochs": Epochs, 
                     "batch_size": Batch,
                     "architecture": "RNN",
                     "dataset": file_name,
                     "vocab_Orignal": len(vocab),
                     "vocab_Max": vocab_len_highest,
                     "activation": "softmax",
                     "optimizer": "Adam",
                     "loss": "sparse_categorical_crossentropy",
                     "metric": "accuracy",
                     })


WANDB_DISABLE_CODE= "true"

In [ ]:
print(vocab)

In [ ]:
index_to_word = {i: word for i, word in enumerate(vocab)}
word_to_index = {word: i for i, word in index_to_word.items()}

In [ ]:
def normalize(s):
    chars = string.punctuation
    for c in tqdm(chars):
        s = s.replace(c," "+c+" ")
    s = s.lower()
    return s

In [ ]:
def accept(words):
    for word in tqdm(words):
        if(not word in vocab):
            return False
    return True

In [ ]:
def text_to_index(s):
    s = normalize(s)
    s_words = s.split()
    if (not accept(s_words)):
        print("Error")
        return ""
    return np.array([word_to_index[word] for word in s_words])

In [ ]:
def word_list_to_index(s_words):
    if (not accept(s_words)):
        print("Error")
        return ""
    return np.array([word_to_index[word] for word in s_words])

In [ ]:
def index_to_text(inds):
    s = ""
    for i in tqdm(range(inds.shape[0])):
        s += index_to_word[inds[i]] + " "
    return s

In [ ]:
def predict(s, numWords):
  s_return = s
  s_ind = text_to_index(s)
  model_new.reset_states()
  
  for i in tqdm(range(s_ind.shape[0])):
    pred = model_new.predict_on_batch(np.array(s_ind[i]).reshape(1,1))
    
  for i in tqdm(range(numWords)):
    next_word_ind = np.argmax(pred)
    s_return += " " + index_to_word[next_word_ind]
    pred = model_new.predict_on_batch(np.array(next_word_ind).reshape(1,1))


  filename = "/content/drive/My Drive/path_to_store_all_stories"
  if not os.path.exists(os.path.dirname(filename)):
    try:
        os.makedirs(os.path.dirname(filename))
    except OSError as exc:
        if exc.errno != errno.EEXIST:
            raise

    f = open(filename, "a")
    f.write("\n")
    f.write("\n")
    f.write("Number of words : \n" + str(numWords))
    f.write("\n")
    f.write("\n")
    f.write("Starting String : \n" + str(s))
    f.write("\n")
    f.write("\n")
    f.write("Story : \n")
    f.write(s_return)
    f.write("\n")
    f.write("\n")
    f.write("\n")
    f.write("\n")
    f.close()
  
  return s_return

In [ ]:
def predictRandom(s, numWords, conf = 1):
  s_return = s
  s_ind = text_to_index(s)
  model_new.reset_states()
  
  for i in tqdm(range(s_ind.shape[0])):
    pred = model_new.predict_on_batch(np.array(s_ind[i]).reshape(1,1))
    
  for i in tqdm(range(numWords)):
    pred_new = np.power(pred[0], conf)
    pred_new = pred_new / np.sum(pred_new)
    
    next_word_ind = np.random.choice(np.arange(vocab_len_highest), p = pred_new)
    
    s_return += " " + index_to_word[next_word_ind]
    pred = model_new.predict_on_batch(np.array(next_word_ind).reshape(1,1))
    
    
  filename = "/content/drive/My Drive/path_to_store_all_stories"
  if not os.path.exists(os.path.dirname(filename)):
    try:
        os.makedirs(os.path.dirname(filename))
    except OSError as exc:
        if exc.errno != errno.EEXIST:
            raise

    f = open(filename, "a")
    f.write("\n")
    f.write("\n")
    f.write("Number of words : \n" + str(numWords))
    f.write("\n")
    f.write("\n")
    f.write("Starting String : \n" + str(s))
    f.write("\n")
    f.write("\n")
    f.write("Story : \n")
    f.write(s_return)
    f.write("\n")
    f.write("\n")
    f.write("\n")
    f.write("\n")
    f.close()
  
  return s_return

In [ ]:
normalize(" irfvjj; dd:kDJJRRJ")

In [ ]:
index_to_text(text_to_index(file_string))

In [ ]:
text_ind = word_list_to_index(text)

In [ ]:
sequence_len = 20
train_x = []
train_y = []

for i in tqdm(range(text_ind.shape[0] - sequence_len - 1)):
    train_x.append(text_ind[i:i+sequence_len])
    train_y.append(text_ind[i+sequence_len])

train_x = np.array(train_x)
train_y = np.array(train_y)

In [ ]:
train_x.shape

In [ ]:
train_y.shape

In [ ]:
index_to_text(train_x[46])

In [ ]:
index_to_text(np.array([train_y[45]]))

In [ ]:
model = keras.Sequential()
model.add(Embedding(vocab_len_highest, 30))
model.add(LSTM(64, return_sequences = True))
model.add(LSTM(128, return_sequences = False))
model.add(Dense(vocab_len_highest, activation = "softmax"))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(), 
              loss = keras.losses.sparse_categorical_crossentropy, 
              metrics=['accuracy'])

In [ ]:
wandb.config.update({"epochs": Epochs, 
                     "batch_size": Batch,
                     "architecture": "RNN",
                     "dataset": file_name,
                     "vocab_Orignal": len(vocab),
                     "vocab_Max": vocab_len_highest,
                     "activation": "softmax",
                     "optimizer": "Adam",
                     "loss": "sparse_categorical_crossentropy",
                     "metric": "accuracy",
                     "layers" : len(model.layers),
                     })

In [ ]:
filepath="/content/drive/My Drive/"+os.path.splitext(folder_file[2])[0]+"-second-{epoch:02d}-{loss:.4f}.hdf5" # Path to save the model checkpoints
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')


log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

callbacks_list = [checkpoint, WandbCallback(), tensorboard_callback]

In [ ]:
model.load_weights("/content/drive/My Drive/"+os.path.splitext(last_folder_file[2])[0]+"_best_weights_ckpt") # Load last best weights

In [ ]:
start_time = time.time()

model.fit(train_x, train_y, batch_size = Batch, epochs = Epochs, callbacks = callbacks_list)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
time_in_min = (time.time() - start_time)/60
print(time_in_min)

In [ ]:
time_in_min = round(time_in_min, 2)
print(time_in_min)

In [ ]:
time_in_hr = round(time_in_min, 2)/60
print(time_in_hr)

In [ ]:
time_in_hr = round(time_in_hr, 2)
print(time_in_hr)

In [ ]:
wandb.config.update({"fitting_time" : str(time_in_hr)+"h"}) # Optional

In [ ]:
#!kill 3639
%tensorboard --logdir logs/fit

In [ ]:
model.save("/content/drive/My Drive/"+os.path.splitext(folder_file[2])[0]+"_final_model.hdf5")

In [ ]:
model.save("/content/drive/My Drive/Machine Learning Research/output/model/Algorithmia/"+os.path.splitext(folder_file[2])[0]+"_algorithmia_model.h5")

In [ ]:
model_new = keras.Sequential()
model_new.add(Embedding(first_vocab_len_highest, 30, batch_input_shape = (1, 1)))
model_new.add(LSTM(64, return_sequences = True, stateful = True))
model_new.add(LSTM(128, return_sequences = False, stateful = True))
model_new.add(Dense(vocab_len_highest, activation = "softmax"))

In [ ]:
model_new.set_weights(model.get_weights())

In [ ]:
predict("", predict_normal_len)

In [ ]:
predictRandom("", predict_random_len, diversity_random)

In [ ]:
# Optional

trained_model_artifact = wandb.Artifact(
            os.path.splitext(folder_file[2])[0]+"_algorithmia_model.h5", type="model",
            description="",)


trained_model_artifact.add_file("/content/drive/My Drive/Machine Learning Research/output/model/Algorithmia/"+os.path.splitext(folder_file[2])[0]+"_algorithmia_model.h5")
wandb.log_artifact(trained_model_artifact)

In [ ]:
# Optional
wandb.alert(
      title='Run  '+main_folder_name+'  Finished',
      text=f'The Run of file {file_name} is completed in {(time.time() - start_time)} seconds.',
      level=AlertLevel.INFO,
      wait_duration= datetime.timedelta(minutes=0)
    )

In [ ]:
# Optional

wandb.finish()